# OLS Linear Regression


## Introduction:
Linear Regression is the simplest form of machine learning method that can be used to identify underlying **linear** relationships in data. It is of the form:$$ y = w_{0} + w_{1}x_{1}+ w_{2}x_{2}+ .... + w_{n}x_{n}$$

The aim of linear regression is to find the best fit line that linearly separates the data reduces the error while doing so.

## Assumptions of Linear Regression:
1. **Linear Relationship:** Linear regression assumes that there is a linear relationship between the predictor variable i.e. X and the target variable i.e. Y.
2. **Correlation:** There should be no or little correlation between the input variables.
3. **Normal features:** All the input variables should be normally distributed.
4. **Autocorrelation of residuals:** The residuals should not be autocorrelated which means that the the current value of the target should not be dependent on the previous observed value.

## Objective Function:
In ordinary least square linear regression we try to minimize the sum of square of the errors by finding a set of weights. Our objective function is given by 
$$Cost(w)  = \frac{1}{2N} \sum_{i=1}^{N} (y_{i}^{pred} - y_{i}^{true})^{2}$$
$$Cost(w) = \frac{1}{2N} \sum_{i=1}^{N} (\sum_{j=1}^{d} w_{j}x_{ij}  - y_{i}^{true})^{2}$$
where,
* N = Number of data points
* d = dimension of a data point

In order to minimize $Cost(w)$ we use gradient descent method in which we find the gradient of the objective function with respect to the weights given by: $$\frac{\partial Cost(w)}{\partial w} = \frac{1}{N} \sum_{i=1}^{N} (\sum_{j=1}^{d} w_{j}x_{ij}  - y_{i}^{true})x_{ij} $$

## Updating Weights:
We update weights using the below equation:
$$w^{new}_{j} = w^{previous}_{j} - \alpha * \frac{\partial Cost(w)}{\partial w} $$
$$w^{new}_{j} = w^{previous}_{j} - \alpha * \frac{1}{N} \sum_{i=1}^{N} (\sum_{j=1}^{d} w_{j}x_{ij}  - y_{i}^{true})x_{ij}$$

We continue doing these updates until we reach convergence.


# Python Implementation:
Before implementing the code let us first define few functions that we will require.


In [116]:
'Import libraries'
from __future__ import division
import numpy as np
import pandas as pd
from collections import defaultdict
import copy
import time
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler

In [117]:
"Util functions"
def paddingData(data):
    '''
    :param data: Data to be paded
    :return: Padded data with value 1 in the first column
    '''
    return np.c_[np.ones(data.shape[0]), data]

def setWeights(numFeat):
    '''
    :param numFeat: Total number of features in data
    :return: vector of ones of length equal to number of features in data
    '''
    return np.ones(numFeat).reshape((numFeat, 1))

def geterror(true, pred):
    '''
    :param true: true labels
    :param pred: predicted labels
    :return: residual
    '''
    return np.linalg.norm(np.subtract(pred, true))/true.shape[0]

def splitData(test_size,cv, numpoints):
    #This function from sklearn takes the length of the data and test size and returns bootstrapped indices 
    #depending on how many boostraps are required
    '''
    :param test_size: size of the test data required (value between 0 and 1).
    :param cv: Number of re-shuffling.
    :param numpoints: Total number of data points.
    :return: indices of the shuffled splits.
    '''
    ss = ShuffleSplit(n=numpoints, n_iter=cv, test_size=test_size, random_state=32)
    return ss

## 1. Batch Gradient Descent Linear Regression:

In [198]:
class BatchLinearRegression():

    def __init__(self,tol):
        '''
        :param alpha: learning rate
        :param tol: tolerance level for convergence
        '''
        self.tolerance = tol
        self.weights = None

    def fit(self,Xtrain,ytrain):
        start = time.time()
        print "Running Batch Gradient Descent"
        runs = 0
        'Do a padding of one'
        Xtrain = paddingData(Xtrain)
        self.weights = setWeights(Xtrain.shape[1])
        while True:
            runs += 1
            loss = np.dot(Xtrain, self.weights) - ytrain
            gradient = np.dot(Xtrain.transpose(), loss) / Xtrain.shape[0]
            temp = self.weights - (1/runs) * gradient
            step = np.linalg.norm(np.subtract(self.weights, temp))
            self.weights = temp
            #print "Step = ", step
            if step < self.tolerance:
                break
        end = time.time()
        print 'Time taken to fit the data:', end-start
        print 'Number of passes over data:',runs

    def predict(self,Xtest):
        Xtest = paddingData(Xtest)
        return np.dot(Xtest, self.weights)

## 2. Ridge Linear Regression:

In [197]:
class RidgeRegression():
    def __init__(self,tol,Lambda):
        '''
        :param alpha: learning rate
        :param tol: tolerance level for convergence
        :param Lambda: regularization parameter
        '''
        self.tolerance = tol
        self.Lambda = Lambda
        self.weights = None

    def fit(self,Xtrain,ytrain):
        start = time.time()
        print "Running Ridge Regression"
        runs = 0
        'Do a padding of one'
        Xtrain = paddingData(Xtrain)
        self.weights = setWeights(Xtrain.shape[1])
        while True:
            runs += 1
            loss = np.dot(Xtrain, self.weights) - ytrain
            gradient = np.dot(Xtrain.transpose(), loss) / Xtrain.shape[0]
            temp = (1 - 2 * self.Lambda * (1/runs)) * self.weights - (1/runs) * gradient
            step = np.linalg.norm(np.subtract(self.weights, temp))
            self.weights = temp
            #print step
            if step < self.tolerance:
                break
        end = time.time()
        print 'Time taken to fit the data:', end-start
        print 'Number of passes over data:',runs
        
    def predict(self,Xtest):
        Xtest = paddingData(Xtest)
        return np.dot(Xtest, self.weights)

## 3. Lasso Linear Regression:

In [196]:
class LassoRegresion():

    def __init__(self,tol,Lambda):
        '''
        :param tol: tolerance level for convergence
        :param Lambda: regularization parameter
        '''
        self.tolerance = tol
        self.Lambda = Lambda
        self.weights = None

    def fit(self,Xtrain,ytrain):
        start = time.time()
        runs = 0
        print "Running Lasso Regression"
        'Do a padding of one'
        Xtrain = paddingData(Xtrain)
        self.weights = setWeights(Xtrain.shape[1])
        while True:
            runs += 1
            temp = copy.copy(self.weights)
            for i in range(1,Xtrain.shape[1]):
                Xnew = np.delete(Xtrain, [i], axis=1)
                Wnew = np.delete(self.weights, [i], axis=0)
                residual = np.linalg.norm(np.subtract(ytrain,np.dot(Xnew, Wnew)))/Xtrain.shape[0]
                if residual < -self.Lambda/2:
                    self.weights[i] = residual + self.Lambda/2
                elif residual < self.Lambda/2 and residual > -self.Lambda/2:
                    self.weights[i] = 0
                else:
                    self.weights[i] = residual - self.Lambda/2

            step = np.linalg.norm(np.subtract(self.weights, temp))
            #print "Step = ", step
            if step < self.tolerance:
                break
        end = time.time()
        print 'Time taken to fit the data:', end-start
        print 'Number of passes over data:',runs
        
    def predict(self,Xtest):
        Xtest = paddingData(Xtest)
        return np.dot(Xtest, self.weights)

## Load the data:
**Now let us load our data and modify it according to the arguments of our algorithms**

In [189]:
from sklearn.datasets import load_boston
boston = load_boston()
numSamples, numFeat = boston.data.shape
ss = splitData(test_size=0.25,cv=1, numpoints=numSamples)
for train_index, test_index in ss:
    Xtrain = boston.data[train_index, :]
    ytrain = boston.target[train_index].reshape((train_index.shape[0], 1))
    Xtest = boston.data[test_index, :]
    ytest = boston.target[test_index].reshape((test_index.shape[0], 1))

In [190]:
'Normalize the data to zero mean and unit variance'
scalar = StandardScaler()
Xtrain = scalar.fit_transform(Xtrain)
Xtest = scalar.transform(Xtest)

## Running our algorithms:
I have also compared the performance of these three algorithms with native sklearn algorithms:

In [204]:
'Import sklearn regressors'
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [205]:
'Run Batch Gradient Descent'
batchreg = BatchLinearRegression(tol=0.0001)
batchreg.fit(Xtrain, ytrain)
pred = batchreg.predict(Xtest)
pred = pred.reshape((pred.shape[0], 1))
print 'Error:',geterror(ytest, pred)

reg = LinearRegression()
reg.fit(Xtrain,ytrain)
pred = reg.predict(Xtest)
print '\nError from running sklearn:',geterror(pred,ytest)


Running Batch Gradient Descent
Time taken to fit the data: 0.0514540672302
Number of passes over data: 2249
Error: 0.459311346147

Error from running sklearn: 0.453497809748


In [206]:
'Run Lasso Regression'
lass = LassoRegresion(tol = 0.0001,Lambda=0.001)
lass.fit(Xtrain,ytrain)
pred = lass.predict(Xtest)
pred = pred.reshape((pred.shape[0], 1))
print 'Error:',geterror(ytest, pred)

reg = LassoLars(alpha=0.001)
reg.fit(Xtrain,ytrain)
pred = reg.predict(Xtest)
print '\nError from running sklearn:',geterror(pred,ytest)

Running Lasso Regression
Time taken to fit the data: 0.00445985794067
Number of passes over data: 4
Error: 2.17753026667

Error from running sklearn: 11.8773268599


In [208]:
'Run Ridge Regression'
rid = RidgeRegression(tol=0.0001,Lambda=0.001)
rid.fit(Xtrain,ytrain)
pred = rid.predict(Xtest)
pred = pred.reshape((pred.shape[0], 1))
print 'Error:',geterror(ytest, pred)

reg = Ridge(tol=0.0001,alpha=0.001)
reg.fit(Xtrain,ytrain)
pred = reg.predict(Xtest)
print '\nError from running sklearn:',geterror(pred,ytest)

Running Ridge Regression
Time taken to fit the data: 0.0685868263245
Number of passes over data: 2219
Error: 0.459630180955

Error from running sklearn: 0.453498270648
